# Music Generation using LSTM neural network

This notebook walk through the process of building Music Generation model based on LSTM using Keras.

## Imports

In [32]:
import tensorflow as tf
import music21 as m21
import numpy as np
import glob

## Preprocessing song dataset

I'm using different piano melodies in MIDI format mostly from Final Fantasy soundtracks.

### Loading

In [33]:
DATA_PATH = '../data/'

Here you can see the code that parse all MIDI files and makes array with all the notes and chords from them.

In [34]:
notes = []

for file in glob.glob(DATA_PATH + '*.mid'):
    midi = m21.converter.parse(file)
    notes_to_parse = None

    parts = m21.instrument.partitionByInstrument(midi)
    if parts:
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
        if isinstance(element, m21.note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, m21.chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

### Shaping data

In [35]:
SEQ_LEN = 100

Get all unique pitchnames.

In [51]:
pitchnames = sorted(set(notes))
len(pitchnames)

326

Create a note_to_int dictionary to map pitches to integer.

In [52]:
note_to_int = {note:number for number, note in enumerate(pitchnames)}
note_to_int[pitchnames[52]]

52

Create input sequences and the corresponding outputs.

In [62]:
network_input = []
network_output = []

for i in range(0, len(notes) - SEQ_LEN):
    input_seq = notes[i:i + SEQ_LEN]
    output_note = notes[i + SEQ_LEN]
    network_input.append([note_to_int[char] for char in input_seq])
    network_output.append(note_to_int[output_note])

Reshape the input into a format compatible with LSTM layers.

In [64]:
network_input = np.reshape(network_input, (len(network_input), SEQ_LEN, 1))

Normalize input and one-hot encode the output.

In [65]:
network_input = network_input / float(len(pitchnames))
network_output = tf.keras.utils.to_categorical(network_output)